In [8]:
import sys
sys.path.append("../")

from uuid import uuid4
from torch import randn
from fgi import *

In [9]:
def generate():
    return str(uuid4())

In [37]:
class Person:
    def __init__(self, gender : bool, name : str, age : int, *args, **kwargs):
        self.gender = gender
        self.name = name
        self.age = age
    
    def __str__(self):
        return f"{self.name}, {self.gender}, {self.age}"
    
    def hello(self, name):
        print(f"{self.name} xin chao {name}")

In [38]:
class FunnyRepresent(SoftRepresentUnit):
    def __init__(self, _id, *args, **kwargs):
        super().__init__(_id, *args, **kwargs)
    
    def forward(self, x, *args, **kwargs):
        return x

In [ ]:
class PersonProblem(NonCodeProblem):
    def __init__(self, _id, *args, **kwargs):
        super().__init__(_id, *args, **kwargs)
        self._represent = RepresentLayer.from_units(
            [FunnyRepresent(generate()) for _ in range(4)], 
            generate()
        )
        self._co_represent = CoRepresentLayer.from_units(
            [CoRepresentUnit(generate(), 4, phi_dim=128) for _ in range(4)], generate()
        )
        self._property = PropertyLayer.from_units(
            [PropertyUnit(generate(), phi_dim=128) for _ in range(4)], generate()
        )
        self._copropery = CoPropertyLayer.from_units(
            [
                ChooseOptions(generate(), 4, options=["male", "female"], property_name="gender", phi_dim=128),
                Regression(generate(), 4, phi_dim=128, property_name="age")
            ],
            generate()
        )

        self._update_additional_infor()

    def forward(self, x, *args, **kwargs):
        x = self._represent(x)
        x = self._co_represent(x)
        x = self._property(x)
        x = self._copropery(x)
        return x
    
    @property
    def _as_object(self):
        return Person
    
    def as_instance(self, x, skip_inference : bool = False, *args, **kwargs):
        if not skip_inference:
            x = self.forward(x)
        data = self._copropery.intepret(x)
        data.update(**kwargs)
        return self._as_object(**data)
    
    def recognize_unknown(self, *args, **kwargs):
        pass

In [56]:
solver = PersonProblem(generate())

In [57]:
x = randn(1, 128)
y = solver(x)

In [43]:
person = solver.as_instance(y, skip_inference=True, name="Haha")

In [44]:
person.hello("Hehe")

Haha xin chao Hehe


In [58]:
class PersonLearner(LightningLearner):
    def __init__(self, problem, *args, **kwargs):
        super().__init__(problem, *args, **kwargs)
    
    def _aggerate_loss(self, y_hat, y, *args, **kwargs):
        return 0

In [59]:
print(solver.metadata)
learner = PersonLearner(solver)

{'type': 'PersonProblem', 'detail': {'_co_represent': {'type': 'CoRepresentLayer', 'components': {'5ff86b35-b214-4109-8cd5-9fa3f467f16f': {'type': 'CoRepresentUnit'}, '4a8f1805-312b-40d0-888e-6a501b0b7ebd': {'type': 'CoRepresentUnit'}, '32c9dd28-3ccf-4b0a-9949-f5d2258ebf14': {'type': 'CoRepresentUnit'}, '7584d36e-baf5-4782-a465-4cbec6969a67': {'type': 'CoRepresentUnit'}}}, '_copropery': {'type': 'CoPropertyLayer', 'components': {'991a2664-5822-453e-8a1e-8dc939cd0d9d': {'type': 'ChooseOptions', 'property': 'gender', 'options': ['male', 'female', 'gay', 'les']}, 'f9a831cb-5f42-42ee-b5b2-163181bc97bd': {'type': 'Regression', 'property': 'age'}}}, '_property': {'type': 'PropertyLayer', 'components': {'9226d905-134b-4a5d-8872-ae8f2c5fa5c5': {'type': 'PropertyUnit'}, '41c2ffb5-b281-49f3-9ece-33fb0df21af4': {'type': 'PropertyUnit'}, '93b96df6-1e2b-46cd-901e-b553ab3aeaeb': {'type': 'PropertyUnit'}, '572b2194-6975-45bc-a93a-3da644054fba': {'type': 'PropertyUnit'}}}, '_represent': {'type': 'Repr